In [0]:
#for setting up on colab

from google.colab import drive
drive.mount('/content/drive/')

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch


In [0]:
#importing libraries
import torch.nn as nn
from torch.utils.data import DataLoader 
import torchvision.transforms as transform #For transformations
import torchvision.datasets as dataset   #dataset is a sub-class of torch.utils.data.Dataset and can be passed torch.utils.data.Dataloader to which can load multiple samples
from torch.autograd import Variable
import torch.nn.functional as F
import copy

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


In [0]:
#initializing batch size and epoch
batch_size=128
epochs = 10

#loading data from MNIST data set in the form of training and testing set
transforms=transform.Compose([transform.ToTensor()])
train_set = dataset.MNIST(root='./data',train=True, 
                           download=True,transform=transforms)
train_loader = DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True,num_workers=4)
test_set = dataset.MNIST(root='./data', train=False,
                        download=True, transform=transforms)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size,shuffle=True,num_workers=4)


In [0]:
#function to add noise
def add_noise(img):
    noise = torch.randn(img.size())*0.4
    noisy_img = img + noise
    return noisy_img

In [0]:
# Encoder 
# torch.nn.Conv2d(in_channels, out_channels, kernel_size,
#                 stride=1, padding=0, dilation=1,
#                 groups=1, bias=True)
# Decoder 
# torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
#                          stride=1, padding=0, output_padding=0,
#                          groups=1, bias=True)
#MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

#one encoding layer consists of one convolution with activation relu and one max pooling operation
#one convolution transpose with activation relu constitutes one decoding layer
#We have used two encoding and decoding layers
class autoencoder(nn.Module):
    def  __init__(self):
        super(autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,padding=1)     #initial dimension 28x28
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3,padding=1)   #dimension remains 28x28 because of padding
        self.conv2_drop = nn.Dropout2d()                           #using drop out to prevent overfitting
        self.fc1 = nn.ConvTranspose2d(128, 64, 3,2,padding=1,output_padding=1) #using convolution transpose to reverse the convolution
        self.fc2 = nn.ConvTranspose2d(64, 1, 3,2,padding=1,output_padding=1)
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) #maxpools with stride of 2 dimension reduces to 14x14
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) #maxpools with stride of 2 dimension reduces to 7x7
        x = F.relu(self.fc1(x)) #tranpose convolution with stride of 2 and kernel 3 upsamples the image to 14x14
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)  #second tranpose convolution with stride of 2 and kernel 3 upsamples the image to 28x28
        return x

In [0]:
net = autoencoder()
use_cuda = True
if use_cuda and torch.cuda.is_available():
    net.cuda()


In [0]:
#optimizer and loss
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.5, momentum=0.9)


In [0]:
for epoch in range(epochs):
    running_loss = 0
    for data in (train_loader):
        image,_=data
        noise_img=add_noise(image).cuda()
        inputs = Variable(noise_img).cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs,Variable(image).cuda())
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
    print('At Iteration : %d   ;  Mean-Squared Error : %f'%(epoch + 1,running_loss/
                                                                (train_set.train_data.size(0)/batch_size)))
print('Finished Training')

In [0]:
#getting an iterable to test our network
dataiter=iter(test_loader)
test_image,_=dataiter.next()
noise_img=add_noise(test_image)
input_image = Variable(noise_img).cuda()
test_image= test_image.view(-1,28,28)
output_image = net(input_image)
output_image=output_image.view(-1,28,28)
input_image=input_image.view(-1,28,28)

In [0]:
input_image.shape

In [0]:
#the shuffle option in test_loader shuffles the images we run the loop for 10 iterations to pick first ten images which are random beccause of shuffling
for i in range(10):
  fig = plt.figure(figsize=(8,8))
  plot=fig.add_subplot(1,3,1)
  img = np.array(test_image[i])
  plot.set_title('Original Image')
  imgplot = plt.imshow(img,cmap='gray')

  plot=fig.add_subplot(1,3,2)
  img = output_image.data
  plot.set_title('Reconstructed Image')
  imgplot = plt.imshow(img[i].cpu().numpy(),cmap='gray')


  plot=fig.add_subplot(1,3,3)
  img = input_image.data
  plot.set_title('Noisy Image')
  imgplot = plt.imshow(img[i].cpu().numpy(),cmap='gray')
  plt.show()

**References:**

For transpose convolution:

https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0 

For basic convolution neural network:

http://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/

For noise in auto encoders:

https://gist.github.com/bigsnarfdude/dde651f6e06f266b48bc3750ac730f80